In [3]:
%%writefile Human_Emotion_Recognition.py

import streamlit as st
import cv2
import numpy as np
from PIL import Image
from deepface import DeepFace
import tempfile

st.title("Human Recognition Application")
st.write("Upload an image or video")

option=st.selectbox('choose an image or video',("image","video"))
                                                
#def a func for human emotion recognition                                                
def Analyze_Emotion(image_or_video):
    try:
        analysis=DeepFace.analyze(image_or_video,actions=['emotion'],enforce_detection=False)
        return analysis[0]['emotion']
    except ValueError as e:
        st.write(f'Error :{e}')
        return None

#to analyze image
if option == "image":
    upload=st.file_uploader("please Upload an image",type=['png','jpg','jpeg'])
    if upload is not None:
        img=Image.open(upload)
        img_array=np.array(img)
        st.image(img_array,caption='Upload Image',channels='RGB',use_column_width=True)
        emotion_scores= Analyze_Emotion(img_array)
        if emotion_scores:
            detected_emotion=max(emotion_scores,key=emotion_scores.get)
            st.write(f'detected_emotion = {detected_emotion}')
        else:
            st.write("NO Face Detected")


if option == "video":
    upload=st.file_uploader("please Upload an video",type=['mp4','mov','avi'])
    if upload is not None:
            with tempfile.NamedTemporaryFile(delete=False) as temp_video:
                temp_video.write(upload.read())
                video_path=temp_video.name
            video = cv2.VideoCapture(video_path)
            frame_rate=30
            frame_count=0
            while video.isOpened():
                ret,frame= video.read()
                if not ret:
                    break
                frame_count +=1
                if frame_count % frame_rate==0:
                    frame_rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                    emotion_scores= Analyze_Emotion(frame_rgb)
                    if emotion_scores:
                        detected_emotion=max(emotion_scores ,key = emotion_scores.get)
                        cv2.putText(frame,detected_emotion, (50,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                    else:
                         detection_emotion='No Detected Face'
                         cv2.putText(frame,detected_emotion, (50,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                    st.image(frame, channels ='BGR')
            video.release()    

Overwriting Human_Emotion_Recognition.py
